<a href="https://colab.research.google.com/github/rodrihgh/MerryXmasEU/blob/main/Wikimedia_Image_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lxml aiohttp asyncio nest_asyncio aiofiles

In [ ]:
import shutil
from lxml import etree
from lxml import html
import aiohttp
import asyncio
import aiofiles
import nest_asyncio
import os
import json

#fix for running in colab
nest_asyncio.apply()

#all categories will be compressed into storeDirectory + download.zip on completion

#ONLY CHNAGE THIS
url = 'https://commons.wikimedia.org/wiki/Category:Saint_Nicholas_in_art'
storeDirectory = '/content/'
checkForCategories = False

file_list = []

#DON'T CHANGE
tasks = []
categories = 0
categoryTasks = []
checkedCategories = []
completed = -1
totalImages = 0
completedImages = 0

async def fetch_page(session, url, cat = ''):
  try:
    async with session.get(url) as resp:
      source = await resp.text()

      dom = html.fromstring(source)

      return [cat, dom]
  except asyncio.TimeoutEraror or aiohttp.ClientConnectorError:
    #print('Timeout')
    return False

async def fetch_images(session, url):
  global totalImages

  dom = await fetch_page(session, url)

  #timeout error
  if dom == False:
    return

  images = dom[1].xpath('*//div[@class="thumb"]//a')
  subcategories = dom[1].xpath('*//div[@class="CategoryTreeItem"]//a')

  if(len(subcategories) > 0 and checkForCategories):
    for category in subcategories:
      if(category not in checkedCategories):
        categoryTasks.append(asyncio.ensure_future(fetch_images(session, 'https://commons.wikimedia.org' + category.attrib['href'])))
        checkedCategories.append(category)
        print('Found category', category.attrib['href'])

  if (len(images) > 0):
    totalImages += len(images)
    print("Found", len(images), "images")
    #download images for each category
    for image in images:
      cat = url.split('Category:')[1]
      tasks.append(asyncio.ensure_future(fetch_page(session, 'https://commons.wikimedia.org' + image.attrib['href'], cat)))

  global completed
  completed += 1

async def main(loop):
  global url
  global completedImages
  global file_list

  async with aiohttp.ClientSession(loop=loop) as session:
    await fetch_images(session, url)

    #fix to resolve finding all categories first
    while True:
      await asyncio.gather(*categoryTasks)

      #check if images have been found on all category pages
      if(completed == len(categoryTasks)):
        break

    pages = await asyncio.gather(*tasks)
    for page in pages:
      #timeout error
      if(page == False):
        continue

      cat = page[0]
      source = page[1]
       
      #print(cat, source.xpath('*//div[@class="fullImageLink"]//img')[0].attrib['src'])
      imgURL = source.xpath('*//div[@class="fullImageLink"]//img')[0].attrib['src']

      filename = imgURL.split('/')[-1]
      #TODO: save images into category folders
      async with session.get(imgURL) as resp:
        if resp.status == 200:
            if(os.path.isdir(storeDirectory + cat + '/') == False):
              os.mkdir(storeDirectory + cat + '/')
            fname = storeDirectory + cat + '/' + filename
            try:
              f = await aiofiles.open(fname, mode='wb')
              await f.write(await resp.read())
              await f.close()
              completedImages += 1
              file_list.append(imgURL)
              print(completedImages, '/', totalImages)
            except OSError:
              pass

  #create zip file to download
  shutil.make_archive(storeDirectory + 'download', 'zip', storeDirectory)

#main event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))

print(json.dumps(file_list, indent=0))

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
